In [ ]:
from sklearn import datasets
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
%matplotlib inline
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from math import exp

In [ ]:
#простая функция расстояния высчитывающая расстояние между 2-мя одноэлементными кластерами
#представленными в виде 2х списков координат (листов)
def d(l1, l2):
    dist = 0
    for i in range(len(l2)):
        dist += abs(l1[i] - l2[i]) ** 2
    return sqrt(dist)

#Функция Коши для размытия степеней принадлежностей точек на нулевом шаге.
def Cauchy(x, c, alpha):
    return 1.0 / (d(c, x) ** 2 + alpha)

#Функция Гаусса для размытия степеней принадлежностей точек на нулевом шаге.
def Gauss(x, c, alpha):
    return exp(1) ** (d(c, x) ** 2 / 2 / alpha * -1)

In [ ]:
#построение матрицы расстояний по центрам кластеров используя частное функций Коши
def build_U(C, X, alpha):
    U = []
    for row in C:
        U.append([-1 for elem in X])
    for i in range(len(C)):
        for j in range(len(X)):
            sumCauchy = 0
            for k in range(len(C)):
                sumCauchy += Cauchy(C[k][:-2], X[j], alpha)
            U[i][j] = Cauchy(C[i][:-2], X[j], alpha) / sumCauchy
    return U

In [ ]:
#различные метрики для нахождения расстояния между кластерами используя центроидный метод.
def abslt(x, y):
    return abs(x - y)

def sqr(x, y):
    return (x - y) * (x - y)

def wgt(x, y):
    return (x - y) * (x + y)

In [ ]:
#пытаемся найти номера двух наипохожих кластеров. К сожалению эта задача очень дорогая по отношению к кол-ву кластеров.
def find_similar_clusters(C):
    similar_cluster_1 = -1
    similar_cluster_2 = -1
    delta = 100000
    dist = 0
    for i in range(len(C)):
        for k in range(len(C)):
            if (i < k):
                temp_delta = 0
                for j in range(len(U[i])):
                    temp_delta += sqr(U[i][j], U[k][j])
                if (temp_delta < delta):
                    delta = temp_delta
                    similar_cluster_1 = C[i][-2]
                    similar_cluster_2 = C[k][-2]
                    dist = d(C[i][:-2], C[k][:-2])
    return [similar_cluster_1, similar_cluster_2, dist]

In [ ]:
#сливаем два центра в один с перерасчётом
def merging_similar_clusters(C, similar_cluster_1, similar_cluster_2, number):
    new_C = []
    for i in range(len(C)):
        if (C[i][-2] != similar_cluster_1 and C[i][-2] != similar_cluster_2):
            new_C.append(C[i].tolist())
        elif (C[i][-2] == similar_cluster_1):
            index_of_similar_cluster_1 = i
        elif (C[i][-2] == similar_cluster_2):
            index_of_similar_cluster_2 = i
    new_center = []
    sum_u_1 = 0
    sum_u_2 = 0
    for j in range(len(X)):
        sum_u_1 += U[index_of_similar_cluster_1][j];
        sum_u_2 += U[index_of_similar_cluster_2][j];
        
    for j in range(len(C[0]) - 2):
        new_center.append((C[index_of_similar_cluster_1][j] * sum_u_1 + C[index_of_similar_cluster_2][j] * sum_u_2) / (sum_u_1 + sum_u_2))
    new_center.append(number)
    new_center.append(C[index_of_similar_cluster_1][-1] + C[index_of_similar_cluster_2][-1])
    
    new_C.append(new_center)
    C = np.array(new_C)
    return C

In [ ]:
alpha = 1

#dataset
iris = datasets.load_iris()
X = iris.data[:, 2:]

bol = 1
count = 0
step_numbers = []

C = X
C = C.tolist()
for i in range(len(C)):
    C[i].append(i)
    C[i].append(1)
U = build_U(C, X, alpha)
step = 0

C = np.array(C)
X_clusters = C[:,0]
Y_clusters = C[:,1]

plt.figure(figsize=(17, 10))
plt.scatter(X_clusters, Y_clusters, color='r')
#plt.savefig('150_irisez_on_flat_surface.jpg')

In [ ]:
Matr = []
number = len(C)
prev_dist = -1
while (len(C) > 1):
    step += 1
    temp_row = []

    U = build_U(C, X, alpha)

    similars = find_similar_clusters(C)
    dist = similars[2]

    temp_row.append(similars[0]) # number of 1st similar cluster
    temp_row.append(similars[1]) # number of 2nd similar cluster
    temp_row.append(similars[2]) # distance between them

    C = merging_similar_clusters(C, similars[0], similars[1], number)

    temp_row.append(C[-1][-1])
    Matr.append(temp_row)        # amount of original dots in new-merged cluster

    print ' step = ', step, '\t|', '\t', "%3d" % (similars[0]), '%3d' % (similars[1]), 'merged into', '%3d' % (number), '\t|', ' \t distance was = ', dist   

    number += 1


In [ ]:
plt.figure(figsize=(18, 23))
axes = plt.gca()

#Set x scale
axes.set_xscale('linear', linthreshx=10)

R = dendrogram(Matr,leaf_font_size=10, orientation='right',distance_sort='ascending')
plt.show()
#plt.savefig('wine_dendro_gauss_sqr_1.jpg')